In [2]:
import checklist
import numpy as np
import spacy
from checklist.editor import Editor
editor = Editor()

In [3]:
# !python -m spacy download en_core_web_sm

In [10]:
# Custom Utils for this Notebook
show_random = lambda ret: np.random.choice(ret.data, 3)
editor.lexicons.keys()

dict_keys(['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'country_city', 'male_from', 'female_from', 'last_from'])

In [5]:
# Checklist Example of Editor filled with Lexicon Values

ret = editor.template('{first_name} is {a:profession} from {country}.',
                       profession=['lawyer', 'doctor', 'accountant'])
show_random(ret)

array(['Bobby is an accountant from Malawi.',
       'Cynthia is a lawyer from Lithuania.',
       'Ian is a lawyer from Chile.'], dtype='<U67')

In [6]:
# Checklist Example of Editor filled with a BERT-based mask
ret = editor.template('This is {a:adj} {mask}.',  
                      adj=['good', 'bad', 'great', 'terrible'])
show_random(ret)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


array(['This is a bad song.', 'This is a bad test.',
       'This is a good pick.'], dtype='<U32')

In [14]:
# Define Our Banking Dictionary & Universe
account_types = [
    "savings",
    "salary",
    "ADIB",
    "Ghina",
    "current",
    "recurring deposit",
    "fixed deposit",
    "loan",
    "demat",
    "checking",
    "",
]
qualifiers = ["I", "we"]
loan_types = ["car", "travel", "personal", "student", "education", "housing", "home"]
debt_synonyms = ["financing", "debt", "loan", "finance"]
card_synonyms = ["card", "covered card", "credit card", "debit card", "atm card"]
card_synonyms += [""] * len(card_synonyms)
card_brands = [
    "Smiles",
    "Emirates",
    "Cashback",
    "Skywards",
    "Rewards",
    "Etihad",
    "Skyward",
    "Booking.com",
    "Spice",
    "amex",
    "diners",
    "discover",
    "jcb",
    "jcb15",
    "jcb16",
    "maestro",
    "mastercard",
    "visa",
    "visa13",
    "visa16",
    "visa19",
]
card_brands += [""] * len(card_brands)

fees_synonyms = ["charges", "fees", "cost", "price"]
balance_synonyms = ["balance", "amount", "figure", "money"]
account_info_terms = [
    "features",
    "interest rates",
    "details",
]
debt_info_terms = [
    "amount due",
    "due amount",
    "payment date",
    "installment",
]
action = ["apply", "get", "open"]
close = ["nearest", "nearby"]
find_close = ["how close is the", "where is the", "need to go to", "ADIB"]
find_what = [
    "ATM",
    "branch",
    "branches",
    "loan agent",
    "cash deposit machine",
    "cheque deposit machine",
    "cash withdrawal",
    "ccdm",
    "cash withdraw machine",
]
inform = [
    "What is my",
    "show me my",
    "find",
    "can you show me the",
    "show",
    "my",
    "show me the",
    "looking for my",
    "Check",
    "Remaining",
    "I want to know",
    "What is",
    "get",
    "Tell me",
    "Where is your",
    "What are the available",
]

Operation | Account | Card |
---|---|---|
Status Check| Yes| Yes|
Fees Check| Yes| Yes|
Get Credentials| Yes| Yes|
Update Credentials| Yes| Yes|
Balance/Dues Check| Yes| Yes|
Apply For| Yes|Yes|
Transfer Money| Yes| No|
Replace/Cancel/Report Missing| No| Yes|

In [15]:
# Sentence Templates with Intent Labels
STATUS_CHECK = "status_check"
FEES_INQUIRY = "fees_inquiry"
GET_CREDENTIALS = "get_credentials"
CHANGE_CREDENTIALS = "change_credential_info"
ACCOUNT_ACTIVITY = "get_account_activity" # added templates
ACCOUNT_BALANCE = "get_account_balance"
DUES_CHECK = "get_pending_dues"
ACTIVATE = "activate" # card or account or some specific feature
REQUEST_CARD_ACTION = "replace_cancel_report_missing_card"
REQUEST_INFO_INTENT = "ask_for_info"
APPLY_FOR = "apply_for" # added templates

# 'Banking_Change_Credential_Info': 2100,
# 'Banking_Card_Request': 2000,
# 'Banking_Fee_Inquiry': 1824,
# 'Banking_Get_Acc_Status': 1800,
# 'Banking_Get_Card_Request_Status': 1714,
# 'Banking_Replace_Card': 1629,
# 'Banking_Cancel_Card': 1550,
# 'Banking_Activate_Card': 1548,

# 'Banking_Get_Account_Balance': 1500,
# 'Banking_Get_Secure_Info': 1500,
# 'Banking_Get_Account_Details': 1200,
# 'Banking_Get_Card_Status': 802,

# 'Banking_Get_Account_Number': 500,
# 'Banking_Get_Address': 500,
# 'Banking_Get_Email': 500,
# 'Banking_Get_Contact': 500,
# 'Banking_Get_Card_Number': 401,
# 'Banking_Get_Username': 400,
# 'Banking_Get_Card_Details': 200,

# 'Banking_Change_Password': 400,
# 'Banking_Change_Address': 360,
# 'Banking_Change_Passcode': 401,
# 'Banking_Card_Number': 25,
# 'Banking_Password': 20,
# 'Banking_Passcode': 11,
# 'Banking_Email': 9

In [ ]:
templates_info_banking = [
    {
        "templates": "{find_close} {mask} {close} {find_what}",
        "labels": REQUEST_INFO_INTENT,
    },
    {
        "templates": "{inform} {account_types} {account_info_terms}",
        "labels": REQUEST_INFO_INTENT,
    },
    {"templates": "{inform} {cards} {debt_info_terms}", "labels": REQUEST_INFO_INTENT},
    {"templates": "{loan_types} {debt_synonyms}", "labels": REQUEST_INFO_INTENT},
    {
        "templates": "{account_types} account {account_info_terms}",
        "labels": REQUEST_INFO_INTENT,
    },
]

In [ ]:
# status check for both cards and account
status_check_templates = [
    {
        "templates": "please assist me to search me my {card_synonyms} status",
        "labels": STATUS_CHECK,
    },
    {
        "templates": "please assist me to search me my {account_types} status",
        "labels": STATUS_CHECK,
    },
    {
        "templates": "help me to obtain me {card_brands} {card_synonyms} status",
        "labels": STATUS_CHECK,
    },
    {"templates": "can you prepare my {card_brands} {card_synonyms} status", "labels": STATUS_CHECK},
    {"templates": "How can i be displayed card status?", "labels": STATUS_CHECK},
    {
        "templates": "please can you help me to come with me {card_brands} {card_synonyms} status",
        "labels": STATUS_CHECK,
    },
    {"templates": "please provide me {card_synonyms} status", "labels": STATUS_CHECK},
    {"templates": "assist to find my {card_brands} status", "labels": STATUS_CHECK},
    {"templates": "overcome me {card_synonyms} status", "labels": STATUS_CHECK},
    {"templates": "display {card_brands} {card_synonyms} status", "labels": STATUS_CHECK},
    {"templates": "give card status", "labels": STATUS_CHECK},
    {"templates": "get me status of card", "labels": STATUS_CHECK},
    {"templates": "catch status of credit card", "labels": STATUS_CHECK},
    {"templates": "display status of my {card_brands} {card_synonyms}", "labels": STATUS_CHECK},
    {"templates": "provide status of {card_brands} {card_synonyms}", "labels": STATUS_CHECK},
    {"templates": "provide status of {card_synonyms}", "labels": STATUS_CHECK},
    {"templates": "show status of {card_brands} {card_synonyms}", "labels": STATUS_CHECK},
    {
        "templates": "assist me to provide whether my account is cancelled",
        "labels": STATUS_CHECK,
    },
    {
        "templates": "can you help me to give whether account is on",
        "labels": STATUS_CHECK,
    },
    {
        "templates": "look through my account and tell me whether account is disabled",
        "labels": STATUS_CHECK,
    },
    {"templates": "obtain my account status", "labels": STATUS_CHECK},
    {
        "templates": "help me to obtain whether or not account is activate",
        "labels": STATUS_CHECK,
    },
    {
        "templates": "see through and tell me whether its is open",
        "labels": STATUS_CHECK,
    },
    {
        "templates": "assist to overcome with whether or not account is revoked",
        "labels": STATUS_CHECK,
    },
    {"templates": "assist me to display my account status", "labels": STATUS_CHECK},
    {
        "templates": "can you please look through and tell me whether my account is turn off",
        "labels": STATUS_CHECK,
    },
]

In [ ]:
fees_inquiry_templates = [
    {
        "templates": "{account_types} account {account_info_terms}",
        "labels": REQUEST_INFO_INTENT,
    },
]
apply_for_templates = [
    {"templates": "{action} {card_brands} {cards}", "labels": APPLY_FOR},
    {
        "templates": "How {verb:mask} {qualifiers} {action} {pron:mask} {card_brands} {cards}",
        "labels": APPLY_FOR,
    },
    {"templates": "{action} {account_types} account", "labels": APPLY_FOR},
    {"templates": "{mask} {mask} open {account_types} account", "labels": APPLY_FOR},
]
account_balance_templates = [
    {
        "templates": "can you help me to obtain me Account number balance",
        labels: ACCOUNT_BALANCE,
    },
    {"templates": "How can i be provided bal of XX2469?", labels: ACCOUNT_BALANCE},
    {"templates": "get my {card_synonyms} bal", labels: ACCOUNT_BALANCE},
    {
        "templates": "please assist to give my balance of {card_synonyms}",
        labels: ACCOUNT_BALANCE,
    },
    {"templates": "search me my {account_types} account bal", labels: ACCOUNT_BALANCE},
    {"templates": "show bal of {account_types} a/c", labels: ACCOUNT_BALANCE},
    {"templates": "show bal of {mask} {card_synonyms} a/c", labels: ACCOUNT_BALANCE},
    {"templates": "give account no bal", labels: ACCOUNT_BALANCE},
    {"templates": "display bal of {account_types} a/c", labels: ACCOUNT_BALANCE},
    {"templates": "prepare balance", labels: ACCOUNT_BALANCE},
]

In [7]:
account_activity_templates = [
    {
        "templates": "Do you know about account activities of my {account_types}?",
        "labels": ACCOUNT_ACTIVITY,
    },
    {"templates": "Transaction history", "labels": ACCOUNT_ACTIVITY},
    {
        "templates": "What is the amount of interest that got credited to my a/c?",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "Please tell me account activities detail",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "I would like to see any recent activity on my account before the interest got added",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "I wanna check my {account_types} summary",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "Help with viewing activity that take place in an a/c",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "From where did I get the latest credit in my {account_types} account",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "When was the transaction processed for my account?",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "I want someone to help me find my a/c transactions and summary for every activity of my account",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "Where to search most recent 2 activity history for a/c?",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "check {mask} activity",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "Can I get statement for transactions in summary?",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "want 2 search accounts transactions",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "Places where activity summary can be obtained",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "assist to provide me my a/c previous 10 activity summary",
        "labels": ACCOUNT_ACTIVITY,
    },
    {
        "templates": "please help me to {mask} my a/c last 12 transactions",
        "labels": ACCOUNT_ACTIVITY,
    },
]

In [8]:
def get_banking_generations(template, nsamples=10):
    """
    Wrapper function for Filling any Sentence Template with Our Banking Universe
    """
    return editor.template(
        **template,
        account_types=account_types,
        qualifiers=qualifiers,
        loan_types=loan_types,
        fees_synonyms=fees_synonyms,
        debt_synonyms=debt_synonyms,
        close=close,
        action=action,
        card_brands=card_brands,
        cards=card_synonyms,
        find_close=find_close,
        inform=inform,
        find_what=find_what,
        debt_info_terms=debt_info_terms,
        account_info_terms=account_info_terms,
        remove_duplicates=True,
        nsamples=nsamples,
        meta=True,
    )

In [9]:
filled_in_info = get_banking_generations(templates_info_banking[-1], nsamples=3)
filled_in_info.data, filled_in_info.meta, filled_in_info.labels

(['fixed deposit account balance',
  'ADIB account charges',
  'current account fees'],
 [{'account_types': 'fixed deposit', 'account_info_terms': 'balance'},
  {'account_types': 'ADIB', 'account_info_terms': 'charges'},
  {'account_types': 'current', 'account_info_terms': 'fees'}],
 ['ask_for_info', 'ask_for_info', 'ask_for_info'])

In [10]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR

editor = Editor()

nlp = spacy.load("en_core_web_sm")

dataset = [
    "This was a very nice movie directed by John Smith.",
    "Mary Keen was brilliant.",
    "I hated everything about this.",
    "This movie was very bad.",
    "I really liked this movie.",
    "just bad.",
    "amazing.",
]
t1 = Perturb.perturb(dataset, Perturb.add_typos)
t2 = Perturb.perturb([x[1] for x in t1.data], Perturb.contractions)

In [11]:
combined_data = [x[1] for x in t1.data] + [x[1] for x in t2.data]

In [12]:
combined_data

['This was a very nice movie directe dby John Smith.',
 'Mary Keen was brilliatn.',
 'I hated everything about htis.',
 'This movie was veryb ad.',
 ' Ireally liked this movie.',
 'jus tbad.',
 'amaizng.']